$\newcommand{\set}[2]{\big\{#1\,\ {\large:}\ \,#2\big\}}
\newcommand{\eps}{\varepsilon}
\newcommand{\abs}[1]{\left\vert#1\right\vert}
\newcommand{\ceil}[1]{\left\lceil#1\right\rceil}
\newcommand{\floor}[1]{\left\lfloor#1\right\rfloor}
\DeclareMathOperator{\sinal}{sinal}
$
# Os métodos da secante e da posição falsa

## $ \S 1 $ Descrição do método da posição falsa

Suponha que a função real contínua $ f $ troque de sinal dentro do intervalo $ [a, b] $. Assim como o método da bissecção, o **método da posição falsa** começa com as duas estimativas $ a $ e $ b $ para o zero. Mas em vez de tomar a próxima estimativa como o ponto médio destes, construímos a reta pelos pontos
$$
\big(a, f(a)\big) \quad \text{e} \quad \big(b,f(b)\big)
$$
e consideramos o ponto onde ela cruza o eixo-$x$, definindo $ \xi $ como sua primeira coordenada. Este seria exatamente o zero de $ f $ caso ela fosse linear, mas em geral isto não acontecerá. Portanto há três possibilidades:
* Se $ f(\xi) = 0 $, então $ \xi $ é um zero de $ f $ e podemos terminar.
* Se $ \sinal\big(f(\xi)\big) \neq \sinal\big(f(a)\big) $, então $ f $ troca de sinal em $ [a, \xi] $. Neste caso fazemos $ b \leftarrow \xi $.
* Se $ \sinal\big(f(\xi)\big) = \sinal\big(f(a)\big) $, então $ f $ troca de sinal em $ [\xi, b] $. Neste caso fazemos $ a \leftarrow \xi $.

Nos dois últimos casos repetimos o procedimento usando o novo intervalo $ [a, b] $ em lugar do original e assim sucessivamente, até que seja satisfeito o critério de parada. Observe que em cada iteração temos a garantia da existência de um zero dentro do intervalo sob consideração, pois $ f $ troca de sinal aí.


![Método da posição falsa](fig_2-1_posicao_falsa_1.png "Title")

## $ \S 2 $ Fórmula para a próxima estimativa no método da posição falsa

A reta por dois pontos $ (x_0, y_0) $ e $ (x_1, y_1) $ é caracterizada pela igualdade da inclinação do segmento que liga $ (x_1, y_1) $ a $ (x_0, y_0) $ e a do segmento ligando um ponto $ (x, y) $ qualquer sobre a reta a $ (x_0, y_0) $. Em símbolos:
$$
\frac{y - y_0}{x - x_0} = \frac{y_1 - y_0}{x_1 - x_0}
$$
Equivalentemente, esta reta é descrita pela equação
$$
y = y_0 + \frac{y_1 - y_0}{x_1 - x_0}\,(x - x_0).
$$

No método da posição falsa utilizamos em cada passo a reta passando por $ \big(a,f(a)\big) $ e $ \big(b,f(b)\big) $, onde $ a $ e $ b $ são as extremidades do intervalo obtido no passo anterior. Substituindo estes valores acima, obtemos a equação
$$
y = f(a) + \frac{f(b) - f(a)}{b - a}\,(x - a).
$$
A próxima estimativa $ \xi $ para o zero é a única que faz esta expressão se anular, ou seja, 
$$
\boxed{\xi = \frac{af(b) - bf(a)}{f(b)-f(a)}}
$$ 

📝 Observe a simetria desta fórmula com respeito a $ a $ e $ b $ e o fato que o denominador é não-nulo pois $ f(a) $ e $ f(b) $ têm sinais opostos por hipótese.

## $ \S 3 $ Implementação do método da posição falsa

Como o método da bissecção e da posição falsa só diferem na fórmula usada para obter a nova estimativa para o zero, as duas implementações são muito parecidas.

In [75]:
def posicao_falsa(f, a, b, eps):
    """
    Utiliza o método da posição falsa para localizar um zero de uma função.
    Entradas:
        * A função real contínua f.
        * As extremidades a e b de um intervalo onde f troca de sinal.
        * A distância máxima tolerada eps do resultado a um zero.
    Saída:
        * Um ponto a distância menor que eps de um zero de f.
    """
    from numpy import sign, log2, ceil
    
    
    iteracoes = 1                         # Contador do número de iterações.
    f_a = f(a)                            # Gravando o valor de f em a.
    f_b = f(b)                            # Gravando o valor de f em b.
    if eps <= 0:
        raise ValueError("A tolerância deve ser positiva!")
    if f_a == 0:
        return a
    elif f_b == 0:
        return b
    elif sign(f_a) == sign(f_b):          # Verifique que [a, b] contém um zero.
        raise ValueError("A função assume valores de mesmo sinal "
                         "nas extremidades dadas!")
    
    while (b - a) >= eps:
        # Próxima estimativa:
        x = (a * f(b) - b * f(a)) / (f(b) - f(a)) 
        f_x = f(x)                        # Gravando o valor de f em m.
        if f_x == 0:
            print("Encontrado um zero exato!")
            return x
        elif sign(f_a) != sign(f_x):      # O intervalo [a, m] contém um zero.
            b = x                         # Tome o novo b como sendo m.
        else:                             # O intervalo [m, b] contém um zero.
            a = x                         # Tome o novo a como sendo m.
            f_a = f_x
        iteracoes += 1
    
    print(f"Encontrado um zero aproximado:\n{x:12.7f}")
    print(f"após {iteracoes} iterações.")
    print(f"O valor da função neste ponto é:\n{f(x):12.7f}")
    return x

## $ \S 4 $ Problemas

**Problema 1:** Usando o método da posição falsa com precisão de três dígitos decimais:

(a) Encontre uma raiz positiva da equação $ \sin x = \frac{x}{2} $.

(a) Encontre a primeira raiz positiva da equação $ \sin x = \frac{x}{n} $ para $ n = 2, 3, \dots, 20 $.

*Solução:*

**Problema 2:** Calcule com ajuda do computador, mas sem usar a implementação acima, as três primeiras iterações do método da posição falsa para estimar uma raiz das equações abaixo nos intervalos indicados. Esboce também os gráficos das funções utilizadas:

(a) $ \tan x - \frac{1}{1 + x^2} = 0 $, $ 0 \le x \le \frac{\pi}{2} $.

(b) $ x^2 = 2 $, $ 0 \le x \le 2 $.

(c) $ x \ln x = 1 $, $ 1 \le x \le 2 $.

(d) $ \cos x = x $, $ 0 \le x \le \frac{\pi}{2} $.

*Solução:*